In [1]:
import optuna
import torch
import torch.nn as nn
import torch.optim as optim

from hyperparameter_tuning import objective
from model_evaluator import ModelEvaluator
from preprocessor import Preprocessor
from station_traffic_model import StationTrafficModel

In [2]:
DATA_PATH = "../../data/combined_tripdata.csv"
INPUT_SEQUENCE_LENGTH = 30
OUTPUT_SEQUENCE_LENGTH = 1

PATIENCE = 3
NUM_EPOCHS = 20

PERFORM_TUNING = True
NUM_TRIALS = 3
params = {
    'model_type': 'lstm',
    'hidden_size': 128,
    'num_layers': 2,
    'dropout': 0.2,
    'learning_rate': 0.001,
    'batch_size': 64,
    'weight_decay': 0.0001
}

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [5]:
# Preprocessing
preprocessor = Preprocessor()
preprocessor.preprocess_data(DATA_PATH, input_sequence_length=INPUT_SEQUENCE_LENGTH,
                             output_sequence_length=OUTPUT_SEQUENCE_LENGTH)
input_size = preprocessor.train_dataset.tensors[0].shape[2]
output_size = preprocessor.train_dataset.tensors[1].shape[2]

In [6]:
# Hyperparameter Tuning
if PERFORM_TUNING:
    study = optuna.create_study(direction='minimize')
    study.optimize(lambda trial: objective(trial, input_size, output_size,
                                           OUTPUT_SEQUENCE_LENGTH, PATIENCE, NUM_EPOCHS, preprocessor, device),
                   n_trials=NUM_TRIALS)
    print("Best hyperparameters:", study.best_params)
    params = params | study.best_params

[I 2025-02-22 16:22:47,606] A new study created in memory with name: no-name-99947e96-0d3a-4356-add9-15494b0ce1d0


[1/4682]   0%|           [00:00<?]

Epoch [1/20] Train Loss: 1277.5081884050717
Epoch [1/20] Validation Loss: 853.7254851228978


[1/4682]   0%|           [00:00<?]

Epoch [2/20] Train Loss: 1265.4944782081961
Epoch [2/20] Validation Loss: 788.6137128072445


...

Epoch [3/20] Train Loss: 437.25101875306626
Epoch [3/20] Validation Loss: 251.8895429064252


[I 2025-02-22 16:29:04,590] Trial 1 finished with value: 251.88967895507812 and parameters: {'model_type': 'lstm', 'hidden_size': 128, 'num_layers': 3, 'dropout': 0.35000000000000003, 'learning_rate': 0.012784628814533171, 'batch_size': 256, 'weight_decay': 0.0055827167974556065}. Best is trial 1 with value: 251.88967895507812.



Model Evaluation Metrics:
mse: 251.8897
rmse: 15.8710
mae: 8.6187


[1/37454]   0%|           [00:00<?]

Engine run is terminating due to exception: 
[W 2025-02-22 16:29:24,429] Trial 2 failed with parameters: {'model_type': 'lstm', 'hidden_size': 256, 'num_layers': 3, 'dropout': 0.30000000000000004, 'learning_rate': 0.06359711378690865, 'batch_size': 8, 'weight_decay': 1.6085505377630135e-05} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "D:\Repos\aai530_2proj\.venv\Lib\site-packages\optuna\study\_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "C:\Users\Doug\AppData\Local\Temp\ipykernel_30864\3491105001.py", line 4, in <lambda>
    study.optimize(lambda trial: objective(trial, input_size, output_size,
                                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "D:\Repos\aai530_2proj\src\station_traffic_model\hyperparameter_tuning.py", line 25, in objective
    model.train_model(criterion, optimizer, train_loader, val_loader, num_epochs, patience, device

In [ ]:
# Training
train_loader, val_loader, test_loader = preprocessor.get_loaders(params['batch_size'])
model = StationTrafficModel(params['model_type'], input_size, params['hidden_size'],
                            output_size,
                            OUTPUT_SEQUENCE_LENGTH, params['num_layers'],
                            params['dropout'])
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=params['learning_rate'], weight_decay=params['weight_decay'])
model.train_model(criterion, optimizer, train_loader, val_loader, NUM_EPOCHS, PATIENCE, device)

In [ ]:
# Evaluation
evaluator = ModelEvaluator(model)
targets, predictions = evaluator.evaluate(test_loader)
metrics = evaluator.calculate_metrics(targets, predictions)
evaluator.plot_predictions(targets, predictions)